In [152]:
from sklearn.preprocessing import StandardScaler, Imputer
import numpy as np
import pandas as pd
import seaborn as sns
from collections import OrderedDict
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from pandas.tools import plotting
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
# from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import matplotlib.pyplot as plt
import re

In [216]:
house_train = pd.read_csv('housetrain.csv', index_col=0)
house_train.head()
house_train = house_train.drop(['Utilities','Electrical','RoofMatl','HouseStyle','Heating','Exterior2nd','Exterior1st',
                               'Condition2','GarageQual','MiscFeature','PoolQC'],1)

In [217]:
house_train['LotFrontage']=house_train['LotFrontage'].fillna(0)
house_train['MasVnrArea']=house_train['MasVnrArea'].fillna(0)
house_train['GarageYrBlt'] = house_train['GarageYrBlt'].fillna(house_train['GarageYrBlt'].mode)

for i in house_train.columns:
    house_train[i] = house_train[i].fillna('NA')

In [218]:
objectdf = house_train.select_dtypes(include=['object'])
objectdf1 = house_train.select_dtypes(exclude=['object'])

objectdfd = objectdf.drop(['GarageYrBlt'],1)

objectdfd.head()

,MSZoning,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,BldgType,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageCond,PavedDrive,Fence,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,RL,Pave,NA,Reg,Lvl,Inside,Gtl,CollgCr,Norm,1Fam,...,Gd,Typ,NA,Attchd,RFn,TA,Y,NA,WD,Normal
2,RL,Pave,NA,Reg,Lvl,FR2,Gtl,Veenker,Feedr,1Fam,...,TA,Typ,TA,Attchd,RFn,TA,Y,NA,WD,Normal
3,RL,Pave,NA,IR1,Lvl,Inside,Gtl,CollgCr,Norm,1Fam,...,Gd,Typ,TA,Attchd,RFn,TA,Y,NA,WD,Normal
4,RL,Pave,NA,IR1,Lvl,Corner,Gtl,Crawfor,Norm,1Fam,...,Gd,Typ,Gd,Detchd,Unf,TA,Y,NA,WD,Abnorml
5,RL,Pave,NA,IR1,Lvl,FR2,Gtl,NoRidge,Norm,1Fam,...,Gd,Typ,TA,Attchd,RFn,TA,Y,NA,WD,Normal


In [219]:
df = pd.get_dummies(objectdfd, columns=objectdfd.columns, drop_first=True)
df

,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,Alley_NA,Alley_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,0,0,1,0,1,1,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0
2,0,0,1,0,1,1,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0
3,0,0,1,0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,0,0,1,0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5,0,0,1,0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
6,0,0,1,0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
7,0,0,1,0,1,1,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0
8,0,0,1,0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
9,0,0,0,1,1,1,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0


In [220]:
df2 = pd.concat([df,objectdf1],axis=1)

In [221]:
df2 = df2.drop(['MasVnrType_NA', 'BsmtCond_NA',
 'Alley_NA',
 'Fence_NA',
 'GarageFinish_NA',
 'GarageCond_NA',
 'BsmtQual_NA',
 'FireplaceQu_NA',
 'BsmtFinType1_NA',
 'GarageType_NA',
 'BsmtFinType2_NA',
 'BsmtExposure_NA'],1)

In [235]:
df2.head()

,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,Alley_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,0,0,1,0,1,0,0,0,1,0,...,0,61,0,0,0,0,0,2,2008,208500
2,0,0,1,0,1,0,0,0,1,0,...,298,0,0,0,0,0,0,5,2007,181500
3,0,0,1,0,1,0,0,0,0,0,...,0,42,0,0,0,0,0,9,2008,223500
4,0,0,1,0,1,0,0,0,0,0,...,0,35,272,0,0,0,0,2,2006,140000
5,0,0,1,0,1,0,0,0,0,0,...,192,84,0,0,0,0,0,12,2008,250000


In [222]:
x=df2.drop(['SalePrice'],1)
y=df2['SalePrice']
train_x , val_x, train_y, val_y = train_test_split(x,y, test_size=0.2,random_state=123)

In [223]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz, DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [224]:
dt = DecisionTreeRegressor(random_state=123, min_samples_split=100, min_samples_leaf=10)
dt.fit(train_x,train_y)
dtpred = dt.predict(val_x)

rf = RandomForestRegressor(random_state=123, min_samples_split=100, min_samples_leaf=10)
rf.fit(train_x,train_y)
rfpred = rf.predict(val_x)

gb = GradientBoostingRegressor(random_state=123)
gb.fit(train_x,train_y)
gbpred = gb.predict(val_x)

In [225]:
print(dt.score(val_x,val_y))
print(rf.score(val_x,val_y))
print(gb.score(val_x,val_y))

0.7445144468198779
0.7498682615697977
0.9060322464440028


In [226]:
house_test = pd.read_csv('housetest.csv', index_col=0)
house_test.head()

house_test = house_test.drop(['Utilities','Electrical','RoofMatl','HouseStyle','Heating','Exterior2nd','Exterior1st',
                               'Condition2','GarageQual','MiscFeature','PoolQC'],1)

In [227]:
print(objectdf.dtypes, '\n')
print(objectdf2.dtypes)

MSZoning         object
Street           object
Alley            object
LotShape         object
LandContour      object
LotConfig        object
LandSlope        object
Neighborhood     object
Condition1       object
BldgType         object
RoofStyle        object
MasVnrType       object
ExterQual        object
ExterCond        object
Foundation       object
BsmtQual         object
BsmtCond         object
BsmtExposure     object
BsmtFinType1     object
BsmtFinType2     object
HeatingQC        object
CentralAir       object
KitchenQual      object
Functional       object
FireplaceQu      object
GarageType       object
GarageYrBlt      object
GarageFinish     object
GarageCond       object
PavedDrive       object
Fence            object
SaleType         object
SaleCondition    object
dtype: object 

MSZoning         object
Street           object
Alley            object
LotShape         object
LandContour      object
LotConfig        object
LandSlope        object
Neighborhood     object


In [228]:
print(house_test.isnull().sum().sort_values(ascending=False)[0:40])

Alley            1352
Fence            1169
FireplaceQu       730
LotFrontage       227
GarageYrBlt        78
GarageFinish       78
GarageCond         78
GarageType         76
BsmtCond           45
BsmtQual           44
BsmtExposure       44
BsmtFinType2       42
BsmtFinType1       42
MasVnrType         16
MasVnrArea         15
MSZoning            4
Functional          2
BsmtHalfBath        2
BsmtFullBath        2
TotalBsmtSF         1
GarageArea          1
KitchenQual         1
SaleType            1
BsmtUnfSF           1
BsmtFinSF2          1
BsmtFinSF1          1
GarageCars          1
RoofStyle           0
ExterCond           0
ExterQual           0
SaleCondition       0
YearRemodAdd        0
YearBuilt           0
OverallCond         0
OverallQual         0
Foundation          0
Condition1          0
Neighborhood        0
LandSlope           0
LotConfig           0
dtype: int64


In [229]:
objectdf2 = house_test.select_dtypes(include=['object'])
objectdf3 = house_test.select_dtypes(exclude=['object'])

In [230]:
for i in objectdf2.columns:
    house_test[i] = house_test[i].fillna('NA')

for i in objectdf3.columns:
    house_test[i] = house_test[i].fillna(0)

In [231]:
df1 = pd.get_dummies(objectdf2, columns=objectdf2.columns, drop_first=True)
df1.head(3)

,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,Alley_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1461,0,1,0,0,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
1462,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1463,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [232]:
dff = pd.concat([df1,objectdf3],axis=1)

In [247]:
dff = dff.fillna(0)
dff = dff.drop(['GarageYrBlt'],1)

In [252]:
dff['SalePrice'] = gb.predict(dff)

In [255]:
dff = dff[['SalePrice']]
dff.to_csv('C:/Users/ziyads059336/Downloads/submission.csv', index=True, header=True)